## Import Packages

In [1]:
from pandas_datareader import data as web
from datetime import date
import math

## Define Dates

In [2]:
# Dates should be in the format 'yyyy/mm/dd'
sdate = '2022/07/05'

# Get todays date
edate = date.today()

## Given some amount of cash £X, what is the most of Stock Y that can be bought at its current price, given a worst case scenario stock price of Z?

In [7]:
# This assumes the currency used to purchase the asset is £GBP and the asset being purchased is in $USD
def amount_of_stock(amount, ticker, worst_case_scenario):    
    
    # Reads data into a dataframe
    df = web.DataReader(ticker, 'yahoo', sdate, edate)

    # need some code to get live conversion rate 
    dollar_amount = 1.2 * amount
   
    # we define the price as the latest closing price
    price = round(df['Close'][-1], 2)
    
    # Calculate the amount required to hold a single share
    maintenance_per_share = price / 10
    
    # Calculate the maximum number of stock we can buy
    amount_of_stock = math.floor((dollar_amount/(price - worst_case_scenario + maintenance_per_share)) * 10) / 10.0
    
    
    required_margin = math.ceil((price / 5) * amount_of_stock) * 1.0
    maintenance_margin = required_margin / 2

    print(f"Stock: {ticker}, Price: ${price}, Amount: {amount_of_stock}")
    print(f"Equity: ${dollar_amount}")
    print(f"Required Margin: ${required_margin}")
    print(f"Maintenance Margin: ${maintenance_margin}")

In [4]:
amount_of_stock(1000, 'SPCE', 4)

Stock: SPCE, Price: $7.32, Amount: 296.1
Equity: $1200.0
Required Margin: $434.0
Maintenance Margin: $217.0


In [5]:
amount_of_stock(100, 'TSLA', 520)

Stock: TSLA, Price: $752.29, Amount: 0.3
Equity: $120.0
Required Margin: $46.0
Maintenance Margin: $23.0


In [6]:
amount_of_stock(1500, 'ARKG', 20)

Stock: ARKG, Price: $37.46, Amount: 84.8
Equity: $1800.0
Required Margin: $636.0
Maintenance Margin: $318.0
